In [1]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [10]:
# Hyper-parameters 
input_size = 57
hidden_size = 50
num_classes = 2
num_epochs = 10
batch_size = 10
learning_rate = 0.01

In [11]:
# Define a customized class for spambase data
class SpamDataset(Dataset):
    """Customized class that pre-processes spam dataset"""
    def __init__(self, csv_file):
        """
        Args:
            csv_file (string): Name of dataset file
        """
        # Read in data
        self.landmarks_frame = pd.read_csv(csv_file, header = None)
        
        # Standardize data
        m = np.mean(self.landmarks_frame.iloc[:,0:57].values)
        st = np.std(self.landmarks_frame.iloc[:,0:57].values)
        self.landmarks_frame.iloc[:,0:57] = (self.landmarks_frame.iloc[:,0:57]-m)/st

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        # Get the features and make it into tensor
        features = self.landmarks_frame.iloc[idx, 0:57].values
        features = torch.from_numpy(features)
        # Get the label
        label = self.landmarks_frame.iloc[idx, -1]
        # Combine features and label into a tuple
        sample = (features,label)
        return sample

In [12]:
# Get dataset
dataset = SpamDataset("spambase.data")

# Specify portion for splitting, shuffle data, set seed
test_split = .2
shuffle_dataset = True
random_seed= 5

# Creating data indices for training and testing splits
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating data samplers and loaders

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=test_sampler)

In [13]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #CUDA is a GPU


In [14]:
# Neural Network Class 
class NeuralNet(nn.Module):
    """ Feedforward Neural Network with 2 hidden layers and ReLU activation function."""
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        torch.manual_seed(5) # For reproducibility
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [15]:
# Set model
model = NeuralNet(input_size, hidden_size, num_classes).double().to(device)

In [16]:
# Set loss function and optimize algorithm
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Set start time
start = time.time()


for epoch in range(num_epochs):
    train_loss, test_loss = [], []
    correct = 0
    total = 0
    # Train the model
    for i, (x, y) in enumerate(train_loader):  
        # Move tensors to the configured device
        x = x.to(device)
        y = y.to(device)
        
        # Forward pass
        outputs = model(x)
        loss = criterion(outputs, y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    
    with torch.no_grad():
        
        # Predict the model
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            outputs = model(x)
            lose = criterion(outputs, y)
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
            test_loss.append(lose.item())

        print ("Epoch:", epoch + 1, ", Training Loss: ", np.mean(train_loss), ", Test loss: ", np.mean(test_loss))

# Set end time        
end = time.time()  
print('Accuracy is: {} %'.format(100 * correct / total))
print('It takes ' +  str(end - start)  + ' seconds to run Feedforward Neural Network algorithm on spambase dataset.')

Epoch: 1 , Training Loss:  0.547222224532694 , Test loss:  0.5069575593968506
Epoch: 2 , Training Loss:  0.508537117995426 , Test loss:  0.5041750824499184
Epoch: 3 , Training Loss:  0.46490058224341724 , Test loss:  0.39653794399620546
Epoch: 4 , Training Loss:  0.4615761704231065 , Test loss:  0.4171171056570944
Epoch: 5 , Training Loss:  0.41145923849504185 , Test loss:  0.3480778366424685
Epoch: 6 , Training Loss:  0.3899358792005264 , Test loss:  0.32533284648955735
Epoch: 7 , Training Loss:  0.37549482206389706 , Test loss:  0.34123323805354555
Epoch: 8 , Training Loss:  0.34931683083690174 , Test loss:  0.3116938689326289
Epoch: 9 , Training Loss:  0.36361249821213243 , Test loss:  0.28028305410154414
Epoch: 10 , Training Loss:  0.3519105223014022 , Test loss:  0.2767392442515016
Accuracy is: 88.91304347826087 %
It takes 25.188838958740234 seconds to run Feedforward Neural Network algorithm on spambase dataset.
